In [20]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-12-02...

import pandas as pd
import time
import sqlite3
import random

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of_7 import properties_of
from my_lib.my_db_functions import df_to_sqlite

In [15]:
# Nustatymai, parametrai:

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'
item_params = 'f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&'

table_version = '_7'
item_producer = 'samsung'

common_prop = {'producer':'samsung','5g':True}

url = f'https://www.varle.lt/{item_type}/?{item_params}'
print(url)

https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&


In [3]:
# Šis fragmentas automatiškai išvalo 'items_df' sukauptus duomenis,
# Pasirengta naujų duomenų įrašymui.

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents','producer','5g'))
# čia bus šios programos rezultatas

In [4]:
# Atidaro Chrome naršyklę,
# reikia patvirtinti "Leisti visus slapukus"!
# Šis fragmentas automatiškai panaikina „draiver“.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

# url = f'https://www.varle.lt/{item_type}/?{item_params}' # iškeliamas į pradžią?

driver.get(url)
# time.sleep(3)
source = driver.page_source
print(f'First page: {url}')

bs = BeautifulSoup(source,'html.parser')
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

First page: https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&
Total pages: 6


In [5]:
# Toliau analizuojamas prekių sąrašo html iš bs=BeautifulSoup().

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

print(f'DF items: {len(items_df)}.')

DF items: 38.


In [6]:
items_df

,id,price,url,price_in_cents,producer,5g
0,37056924,2379€,https://www.varle.lt/mobilieji-telefonai/samsu...,NaN,NaN,NaN
1,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,NaN,NaN,NaN
2,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,NaN,NaN,NaN
3,22867446,2099.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,NaN,NaN,NaN
4,22970524,1989.99€,https://www.varle.lt/mobilieji-telefonai/isman...,NaN,NaN,NaN
5,24423379,1669.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,NaN,NaN,NaN
6,37312882,1539.73€,https://www.varle.lt/mobilieji-telefonai/telef...,NaN,NaN,NaN
7,31784534,1459€,https://www.varle.lt/mobilieji-telefonai/isman...,NaN,NaN,NaN
8,31784528,1459€,https://www.varle.lt/mobilieji-telefonai/isman...,NaN,NaN,NaN
9,31786995,1459€,https://www.varle.lt/mobilieji-telefonai/isman...,NaN,NaN,NaN


In [ ]:
# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

import random

# for page_number in range(2,total_pages+1):
for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?{item_params}p={page_number}'
    driver.get(url)
    time.sleep(random.randint(25,75))
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='last',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# driver.close() # Uždaro Chrome puslapį.

Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=2
Get 76 items. Unique 76 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=3
Get 114 items. Unique 114 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=4
Get 152 items. Unique 152 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=5
Get 190 items. Unique 190 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=6
Get 212 items. Unique 212 items.


,id,price,url,price_in_cents,producer,5g
0,37056924,2379€,https://www.varle.lt/mobilieji-telefonai/samsu...,237900,NaN,NaN
1,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,NaN,NaN
2,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,NaN,NaN
3,22867446,2099.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,209998,NaN,NaN
4,22970524,1989.99€,https://www.varle.lt/mobilieji-telefonai/isman...,198999,NaN,NaN
...,...,...,...,...,...,...
207,34513543,149.30€,https://www.varle.lt/mobilieji-telefonai/telef...,14930,samsung,NaN
208,34513544,145.07€,https://www.varle.lt/mobilieji-telefonai/telef...,14507,samsung,NaN
209,24009958,141.52€,https://www.varle.lt/mobilieji-telefonai/samsu...,14152,samsung,NaN
210,23499397,137.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,13799,samsung,NaN


In [ ]:
# Pridedamas kainos stulpelis centais (int)

import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)

In [9]:
df=items_df

In [13]:
df['producer'] = 'samsung'
df['5g'] = True
df

,id,price,url,price_in_cents,producer,5g
0,37056924,2379€,https://www.varle.lt/mobilieji-telefonai/samsu...,237900,samsung,True
1,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
2,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
3,22867446,2099.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,209998,samsung,True
4,22970524,1989.99€,https://www.varle.lt/mobilieji-telefonai/isman...,198999,samsung,True
...,...,...,...,...,...,...
207,34513543,149.30€,https://www.varle.lt/mobilieji-telefonai/telef...,14930,samsung,True
208,34513544,145.07€,https://www.varle.lt/mobilieji-telefonai/telef...,14507,samsung,True
209,24009958,141.52€,https://www.varle.lt/mobilieji-telefonai/samsu...,14152,samsung,True
210,23499397,137.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,13799,samsung,True


In [16]:
f'{item_type}{table_version}'

'mobilieji-telefonai_7'

In [18]:
# Save to SQLite3
df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{table_version}')

Lentelė 'mobilieji-telefonai_7' sėkmingai įrašyta į DB.


In [21]:
mobilieji_telefonai = properties_of("mobilieji_telefonai",engine="sqlite_file",db_path="./my_scrap_prop_of.db")
mobilieji_telefonai.import_from_df(items_df)

In [23]:
print(mobilieji_telefonai.export_to_df(['5g','price']))

property_id object_id 5g     price
0            14511169  1   729.99€
1            14522358  1   709.99€
2            15087466  1   439.99€
3            15124168  1   511.38€
4            15124169  1   689.99€
..                ... ..       ...
207          37974184  1   249.99€
208          38371553  1   235.99€
209          39080345  1   355.99€
210          39090142  1   280.30€
211          40650587  1  1149.99€

[212 rows x 3 columns]
